<a href="https://colab.research.google.com/github/edtechequity/ml_fairness_toolkit/blob/master/statistical_fairness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Statistical Fairness

This module is intended to provide a quick overview of methods for statistical fairness. We cover three common types, in increasing order of complexity. No single metric will alone ensure fairness, but it can be helpful to see how your model fares across them.
- **Unawareness.**
- **Demographic Parity.**
- **Equalized Odds.**



Statistical fairness is still an active area of research. If interested in a deeper dive, consider this [this tutorial](https://mrtz.org/nips17/#/) offered at NIPS (Neural Information Processing Systems Conference).

This notebook will use the same COMPAS dataset used in the other two modules. You can download it from Kaggle [here](https://www.kaggle.com/danofer/compass/data#propublica_data_for_fairml.csv). The file we'll be using is `propublica_data_for_fairml.csv`.

### Import Dataset

First, let's import a dataset. We recommend using our sample Propublica dataset first, then retrying some of this analysis with your own data.

You can upload your file using the "upload" button in the top left of Google Colab to begin. This code assumes a CSV or Pandas-compliant format.

The code below imports the data from the uploaded file.

In [0]:
import pandas as pd 

df = pd.read_csv('propublica_data_for_fairml.csv')
df[:5]

,Two_yr_Recidivism,Number_of_Priors,score_factor,Age_Above_FourtyFive,Age_Below_TwentyFive,African_American,Asian,Hispanic,Native_American,Other,Female,Misdemeanor
0,0,0,0,1,0,0,0,0,0,1,0,0
1,1,0,0,0,0,1,0,0,0,0,0,0
2,1,4,0,0,1,1,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,1
4,1,14,1,0,0,0,0,0,0,0,0,0


# 1. Unawareness
**Note: this is widely considered an insufficient method for measuring statistical fairness.**

- **Intuition.** Unawareness means that the machine learning model does not take as a feature any protected attributes, like race and gender. 
- **Pros.**
  - Simple and intuitive
  - Often satisfies legal requirements (see [disparate treatment](https://en.wikipedia.org/wiki/Disparate_treatment))
- **Cons.**
  - Fails when the dataset has other features that are highly correlated with protected attributes (e.g., neighborhood and race). Check out our [Dataset notebook](https://github.com/edtechequity/ml_fairness_toolkit/blob/master/dataset.ipynb) for feature correlation.

## 2. Demographic Parity

- **Intuition.** Demographic parity is satisfied when a classifier is equally likely to flag people in two groups (e.g., `African_American` and not).
- **Math.** The probability of being flagged is independent of the protected attribute.

- **Pros.**
  - Fairly intuitive: you want your model to flag people at the same rate irrespective of certain protected categories.
  - Often satisfies legal requirements (see ["the 80% rule"](https://en.wikipedia.org/wiki/Disparate_impact#The_80%_rule))
- **Cons.**
  - If applied without care, it can unfairly penalize certain groups. If older people are less likely to reoffend but a model is constrained by statistical parity, it will either incorrectly flag more of the elderly or incorrectly clear young people.


### Demographic Parity Examples
The code below calculates the rate at which in- and out-groups are flagged. 


- **Example 1: African American people.** The COMPAS algorithm flags African American people 58% of the time, compared to 31% otherwise. This means African Americans are flagged at almost twice the rate of everyone else. This does **not** satisfy "the 80% rule" for demographic parity. 

- **Example 2: Women.** The COMPAS algorithm flags women 41% of the time and men 46% of the time, which is far closer to satisfying demographic parity. This satisfies "the 80% rule."

In [0]:
# The following code will count how many people of certain classes were flagged by the COMPAS algorithm.

def print_positive_rates(df, prediction, feature):
  in_class = df.apply(lambda d: d[feature] == 1, axis=1)
  out_class = df.apply(lambda d: d[feature] == 0, axis=1)
  in_flagged = df.apply(lambda d: d[prediction] == 1 and d[feature] == 1, axis=1)
  out_flagged = df.apply(lambda d: d[prediction] == 1 and d[feature] == 0, axis=1)

  num_in_class = len(in_class[in_class == True].index)
  num_out_class = len(out_class[out_class == True].index)
  num_in_flagged = len(in_flagged[in_flagged == True].index)
  num_out_flagged = len(out_flagged[out_flagged == True].index)

  pct_in_flagged = round(100 * num_in_flagged / num_in_class)
  pct_out_flagged = round(100 * num_out_flagged / num_out_class)

  print('In class flagged at a rate of ', pct_in_flagged, '%.', sep='')
  print('Out class flagged at a rate of ', pct_out_flagged, '%.', sep='')

  lower = min(pct_in_flagged, pct_out_flagged)
  higher = max(pct_in_flagged, pct_out_flagged)
  if lower / higher > .8:
    print ('This passes "the 80% rule" of demographic parity.')
  else:
    print ('This DOES NOT pass "the 80% rule" of demographic parity.')

print('*** Example 1: Demographic Parity Test for African American folks ***')
print_positive_rates(df, 'score_factor', 'African_American')

print('\n*** Example 2: Demographic Parity Test for women ***')
print_positive_rates(df, 'score_factor', 'Female')

*** Example 1: Demographic Parity Test for African American folks ***
In class flagged at a rate of 58%.
Out class flagged at a rate of 31%.
This DOES NOT pass "the 80% rule" of demographic parity.

*** Example 2: Demographic Parity Test for women ***
In class flagged at a rate of 41%.
Out class flagged at a rate of 46%.
This passes "the 80% rule" of demographic parity.


# 3. Equalized Odds
- **Intuition.** Equalized odds is satisfied when a classifier is equally likely to flag people in two groups who didn't deserve to have been flagged. It's different from demographic parity because it takes into account whether each individual eventually reoffends.
- **Math.** The probability of being flagged is independent of the protected category, given the final label (whether they should have been flagged).
- **Pros.**
  - This "fixes" a problem with demographic parity by allowing a model to flag certain groups at higher rates if they are actually more likely to reoffend. For example, you allow a model to flag people under 60 more than people over 60.
  - It creates an incentive for the algorithm maker to reduce errors in the worst-off group first, because the model must perform equally "bad" for all classes.
- **Cons.**
  - This metric doesn't account for societal issues that cause certain groups to be more likely to merit flagging. If there's a societal reason women are less likely to reoffend, using equalized odds does nothing to help men reoffend less. 

### Equalized Odds Examples

The code below calculates the rate at which in- and out-groups who did not eventually reoffend were incorrectly flagged.

- **Example 1: African American people.** The COMPAS algorithm incorrectly flags African American people who didn't go on to reoffend 42% of the time, compared to 20% otherwise. This means innocent African Americans are flagged at over twice the rate of everyone else. This does **not** come close to satisfying equalized odds.

- **Example 2: Women.** The COMPAS algorithm incorrectly flags women who didn't go on to reoffend 30% of the time, the same as men. This does satisfy equalized odds.


In [0]:
def print_conditional_positive_rates(df, prediction, label, feature):
  in_class = df.apply(lambda d: d[feature] == 1 and d[label] == 0, axis=1)
  out_class = df.apply(lambda d: d[feature] == 0 and d[label] == 0, axis=1)
  in_flagged = df.apply(lambda d: d[prediction] == 1 and d[label] == 0 and d[feature] == 1, axis=1)
  out_flagged = df.apply(lambda d: d[prediction] == 1 and d[label] == 0 and d[feature] == 0, axis=1)

  num_in_class = len(in_class[in_class == True].index)
  num_out_class = len(out_class[out_class == True].index)
  num_in_flagged = len(in_flagged[in_flagged == True].index)
  num_out_flagged = len(out_flagged[out_flagged == True].index)

  pct_in_flagged = round(100 * num_in_flagged / num_in_class)
  pct_out_flagged = round(100 * num_out_flagged / num_out_class)

  print('In class flagged at a rate of ', pct_in_flagged, '%.', sep='')
  print('Out class flagged at a rate of ', pct_out_flagged, '%.', sep='')

  lower = min(pct_in_flagged, pct_out_flagged)
  higher = max(pct_in_flagged, pct_out_flagged)
  if lower / higher > .9:
    print ('This passes the equalized odds test.')
  else:
    print ('This DOES NOT pass the equalized odds test.')

print('*** Example 1: Equalized Odds Test for African American folks ***')
print_conditional_positive_rates(df, 'score_factor', 'Two_yr_Recidivism', 'African_American')

print('\n*** Example 2: Equalized Odds Test for women ***')
print_conditional_positive_rates(df, 'score_factor', 'Two_yr_Recidivism', 'Female')

*** Example 1: Equalized Odds Test for African American folks ***
In class flagged at a rate of 42%.
Out class flagged at a rate of 20%.
This DOES NOT pass the equalized odds test.

*** Example 2: Equalized Odds Test for women ***
In class flagged at a rate of 30%.
Out class flagged at a rate of 30%.
This passes the equalized odds test.


# Summary

No statistical fairness metric is perfect, and it's often mathematically impossible to perfectly satisfy all definitions at the same time. However, statistical fairness should be a part of the core metrics that your team monitors internally. This is the only way to ensure that you're creating an equitable product that remains equitable over time. 